In [1]:
#%matplotlib notebook
%matplotlib inline 

%load_ext autoreload
%autoreload 2

import os, sys
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd


sys.path.append("../../pycode/")
#from magus_util import read_nvprof_trace, parse_nvprof_trace, getruntime, sort_dict_by_val, genNNFeat, testFeat
from magus_util import *
from magus_contention import *

# load dedicated execution time

In [2]:
dedicated_app_dd = np.load("../../09_case_study/dedicated_app_runtime.npy").item()

In [3]:
# test 
dedicated_app_dd['rodinia_b+tree']

2326.0

# load the app2metric 

In [4]:
app2metric_dd = np.load("../../07_sim_devid/similarity/app2metric_dd.npy").item()

In [5]:
# test
# app2metric_dd['rodinia_b+tree']
# app2metric_dd['rodinia_btree']

# similarity distance

In [6]:
app2app_sim_dd = np.load('app2app_sim_dd.npy').item()

In [7]:
# test
# app2app_sim_dd['rodinia_b+tree']
# app2app_sim_dd['rodinia_btree']

#  features from traces

In [8]:
app_iobound_dd = np.load('app_iobound_dd.npy').item()

app2newfeat_dd = np.load('app2newfeat_dd.npy').item()

# ground truth: speedup for run2

In [9]:
#
# /magus/09_case_study/testall_greedy/2-groundtruth_cases_from_run2log.ipynb
# 
run2_groundTruth_dd = np.load("../../09_case_study/testall_greedy/app_run2_speedup_dd.npy").item()

In [10]:
#run2_groundTruth_dd['rodinia_b+tree']

# study all the application

In [11]:
def find_large_and_small_jobs(target_app, app2newfeat_dd, appPool):
    target_appdur  = app2newfeat_dd[target_app][0]
    target_prepare = app2newfeat_dd[target_app][1]
    
    large_job_list = []
    small_job_list = []
    total_job_list = []

    for myapp in appPool:
        total_job_list.append(myapp)
        value = app2newfeat_dd[myapp]
        my_appdur  = value[0]
        my_prepare = value[1]
        #print my_appdur, my_prepare
        
        if my_prepare >= target_appdur and my_appdur > target_appdur * 2.0:
            large_job_list.append(myapp)

        if my_appdur <= target_prepare and my_appdur * 2.0 < target_appdur:
            small_job_list.append(myapp)
        
    mylist = []
    
    if large_job_list:  # prefer large job to corun
        mylist = large_job_list
    else:
        if small_job_list:   # if no large job, use the small job list
            mylist = small_job_list
        else:
            mylist = total_job_list  # if there is no large/small job, use the orginial set
    
    
    #print("large_job_list = {} \t small_job_list = {}".format(len(large_job_list), len(small_job_list)))
    
    return mylist
    

In [12]:
def select_opposite_type_app(target_app, job_list, app_iobound_dd):
    app1_type = app_iobound_dd[target_app]
    opp_type_list = []
    for app2 in job_list:
        app2_type = app_iobound_dd[app2]
        if app1_type <> app2_type:
            opp_type_list.append(app2)
    return opp_type_list
            

In [13]:
def find_corun_list(app1, app2newfeat_dd, job_list, DEV_THREAD_MAX):
    #[a1_thd_max, _, a1_reg_max, _, a1_sm_max, _, a1_trans_max, _] =app2newfeat_dd[app1][3:]
    a1_thd_max = app2newfeat_dd[app1][3]
    #print a1_thd_max, a1_reg_max, a1_sm_max, a1_trans_max, "\n"

    avail_thd = DEV_THREAD_MAX - a1_thd_max

    corun_list = []
    if avail_thd > 0:  # for cases, where there are more threads can be launched
        for i in job_list:
            #[threads_max, _, reg_max, _, sm_max, _, trans_max, _] =app2newfeat_dd[i][3:]
            threads_max = app2newfeat_dd[i][3]
            
            #print threads_max, reg_max, sm_max, trans_max
            if threads_max <= avail_thd:
                corun_list.append(i)
    return corun_list

In [14]:
def find_least_sim(app1, app2app_sim_dd, job_list):
    app1_sim_dd = app2app_sim_dd[app1]

    dist_dd = {}
    for app2 in job_list:
        dist_dd[app2] = app1_sim_dd[app2]

    sorted_dist = sorted(dist_dd.items(), key=operator.itemgetter(1))
    #print sorted_dist
    #print sorted_dist[-1]

    # the least similar one is the last, since sorting in non-decreasing order
    return sorted_dist[-1]

In [15]:
appPool = [key for key, _ in app_iobound_dd.iteritems()]
print len(appPool)

79


In [26]:
appPool = [key for key, _ in app_iobound_dd.iteritems()]

count = 0
DEV_THREAD_MAX = 28 * 2048
total_apps = len(appPool)


run2_list = []

for k in xrange(total_apps):
    
    if len(appPool) == 1:
        run2_list.append([appPool[0],None])
        break
    
    if appPool:
        
        #print appPool
        #print len(appPool)
        
        # select the 1st app in appPool
        app1 = appPool[0]

        idx = appPool.index(app1) # update appPool by removing the selected app
        del appPool[idx]

        #=================================#
        # start selection
        #=================================#
        TARGET = None
        CNT = True

        #===================================#
        # step1 : separate the large and small job list
        #===================================#
        #print "Start step1:"
        job_list = find_large_and_small_jobs(app1, app2newfeat_dd, appPool)

        KeepGo = True
        if len(job_list) == 1:
            KeepGo = False
            TARGET = job_list[0]
            #print("step 1 => Found: {}".format(TARGET))

            #
            # update appPool by removing the selected app to corun
            # 
            print("=> run {} with {}".format(app1, TARGET))
            idx = appPool.index(TARGET)
            del appPool[idx]
            
            run2_list.append([app1,TARGET])

        CNT = (CNT and KeepGo)

        #===================================#
        # step2 : prefer opposite type job to corun
        #===================================#

        #print job_list
        if CNT:
            #print "Start step2\n"
            opp_type_list = select_opposite_type_app(app1, job_list, app_iobound_dd)


            if opp_type_list:
                job_list = opp_type_list   # update job_list if there are opposite types

            KeepGo = True
            if len(job_list) == 1:
                KeepGo = False
                TARGET = job_list[0]
                #print("step 2 => Found: {}".format(TARGET))

                #
                # update appPool by removing the selected app to corun
                # 
                print("=> run {} with {}".format(app1, TARGET))
                idx = appPool.index(TARGET)
                del appPool[idx]
                run2_list.append([app1,TARGET])


            CNT = (CNT and KeepGo)

            #===================================#
            # step3 : check the thread limitation
            #===================================#
            if CNT:
                #print "Start step 3\n"
                corun_list = find_corun_list(app1, app2newfeat_dd, job_list, DEV_THREAD_MAX)
                if corun_list:
                    job_list = corun_list

                KeepGo = True
                if len(job_list) == 1:
                    KeepGo = False
                    TARGET = job_list[0]
                    #print("step 3 => Found: {}".format(TARGET))

                    #
                    # update appPool by removing the selected app to corun
                    # 
                    print("=> run {} with {}".format(app1, TARGET))
                    idx = appPool.index(TARGET)
                    del appPool[idx]
                    
                    run2_list.append([app1,TARGET])


                CNT = (CNT and KeepGo)


        #===================================#
        # step4 : similarity analysis
        #===================================#
        if CNT:
            #print "Start step 4\n"
            least_sim = find_least_sim(app1, app2app_sim_dd, job_list) # return value is a tuple (appName, distVal)
            TARGET = least_sim[0]
            #print("step 4 => Found: {}".format(TARGET))

            #
            # update appPool by removing the selected app to corun
            # 
            print("=> run {} with {}".format(app1, TARGET))
            idx = appPool.index(TARGET)
            del appPool[idx]
            
            run2_list.append([app1,TARGET])
    
    #print appPool
    
    #=================================#
    # end of checking appPool
    #=================================#

=> run rodinia_b+tree with cudasdk_FDTD3d
=> run rodinia_heartwall with lonestar_dmr
=> run cudasdk_convolutionFFT2D with rodinia_lavaMD
=> run shoc_lev1BFS with cudasdk_radixSortThrust
=> run parboil_cutcp with lonestar_sssp
=> run rodinia_hybridsort with shoc_lev1md5hash
=> run cudasdk_MCEstimatePiInlineQ with cudasdk_batchCUBLAS
=> run cudasdk_MCEstimatePiInlineP with shoc_lev1GEMM
=> run cudasdk_shflscan with poly_syr2k
=> run rodinia_dwt2d with cudasdk_scan
=> run cudasdk_vectorAdd with cudasdk_eigenvalues
=> run cudasdk_MCEstimatePiP with poly_syrk
=> run cudasdk_MCEstimatePiQ with parboil_lbm
=> run parboil_bfs with cudasdk_MCSingleAsianOptionP
=> run cudasdk_stereoDisparity with lonestar_bh
=> run shoc_lev1fft with poly_correlation
=> run cudasdk_segmentationTreeThrust with cudasdk_binomialOptions
=> run cudasdk_simpleCUBLAS with lonestar_mst
=> run parboil_sgemm with cudasdk_simpleCUFFTcallback
=> run cudasdk_concurrentKernels with cudasdk_reduction
=> run cudasdk_fastWalshTra

In [27]:
run2_list

[['rodinia_b+tree', 'cudasdk_FDTD3d'],
 ['rodinia_heartwall', 'lonestar_dmr'],
 ['cudasdk_convolutionFFT2D', 'rodinia_lavaMD'],
 ['shoc_lev1BFS', 'cudasdk_radixSortThrust'],
 ['parboil_cutcp', 'lonestar_sssp'],
 ['rodinia_hybridsort', 'shoc_lev1md5hash'],
 ['cudasdk_MCEstimatePiInlineQ', 'cudasdk_batchCUBLAS'],
 ['cudasdk_MCEstimatePiInlineP', 'shoc_lev1GEMM'],
 ['cudasdk_shflscan', 'poly_syr2k'],
 ['rodinia_dwt2d', 'cudasdk_scan'],
 ['cudasdk_vectorAdd', 'cudasdk_eigenvalues'],
 ['cudasdk_MCEstimatePiP', 'poly_syrk'],
 ['cudasdk_MCEstimatePiQ', 'parboil_lbm'],
 ['parboil_bfs', 'cudasdk_MCSingleAsianOptionP'],
 ['cudasdk_stereoDisparity', 'lonestar_bh'],
 ['shoc_lev1fft', 'poly_correlation'],
 ['cudasdk_segmentationTreeThrust', 'cudasdk_binomialOptions'],
 ['cudasdk_simpleCUBLAS', 'lonestar_mst'],
 ['parboil_sgemm', 'cudasdk_simpleCUFFTcallback'],
 ['cudasdk_concurrentKernels', 'cudasdk_reduction'],
 ['cudasdk_fastWalshTransform', 'rodinia_gaussian'],
 ['cudasdk_SobolQRNG', 'poly_3dcon

# generate script to run2

In [41]:
def pick_app2run(app_list, app2newfeat_dd, app_iobound_dd, app2app_sim_dd):
    
    appPool = copy.deepcopy(app_list)
    
    DEV_THREAD_MAX = 28 * 2048
    total_apps = len(appPool)

    run2_list = []

    for k in xrange(total_apps):

        if len(appPool) == 1:
            run2_list.append([appPool[0],None])
            break

        if appPool:

            #print appPool
            #print len(appPool)

            # select the 1st app in appPool
            app1 = appPool[0]

            idx = appPool.index(app1) # update appPool by removing the selected app
            del appPool[idx]

            #=================================#
            # start selection
            #=================================#
            TARGET = None
            CNT = True

            #===================================#
            # step1 : separate the large and small job list
            #===================================#
            #print "Start step1:"
            job_list = find_large_and_small_jobs(app1, app2newfeat_dd, appPool)

            KeepGo = True
            if len(job_list) == 1:
                KeepGo = False
                TARGET = job_list[0]
                #print("step 1 => Found: {}".format(TARGET))

                #
                # update appPool by removing the selected app to corun
                # 
                #print("=> run {} with {}".format(app1, TARGET))
                
                idx = appPool.index(TARGET)
                del appPool[idx]

                run2_list.append([app1,TARGET])

            CNT = (CNT and KeepGo)

            #===================================#
            # step2 : prefer opposite type job to corun
            #===================================#

            #print job_list
            if CNT:
                #print "Start step2\n"
                opp_type_list = select_opposite_type_app(app1, job_list, app_iobound_dd)


                if opp_type_list:
                    job_list = opp_type_list   # update job_list if there are opposite types

                KeepGo = True
                if len(job_list) == 1:
                    KeepGo = False
                    TARGET = job_list[0]
                    #print("step 2 => Found: {}".format(TARGET))

                    #
                    # update appPool by removing the selected app to corun
                    # 
                    #print("=> run {} with {}".format(app1, TARGET))
                    idx = appPool.index(TARGET)
                    del appPool[idx]
                    run2_list.append([app1,TARGET])


                CNT = (CNT and KeepGo)

                #===================================#
                # step3 : check the thread limitation
                #===================================#
                if CNT:
                    #print "Start step 3\n"
                    corun_list = find_corun_list(app1, app2newfeat_dd, job_list, DEV_THREAD_MAX)
                    if corun_list:
                        job_list = corun_list

                    KeepGo = True
                    if len(job_list) == 1:
                        KeepGo = False
                        TARGET = job_list[0]
                        #print("step 3 => Found: {}".format(TARGET))

                        #
                        # update appPool by removing the selected app to corun
                        # 
                        #print("=> run {} with {}".format(app1, TARGET))
                        idx = appPool.index(TARGET)
                        del appPool[idx]

                        run2_list.append([app1,TARGET])


                    CNT = (CNT and KeepGo)


            #===================================#
            # step4 : similarity analysis
            #===================================#
            if CNT:
                #print "Start step 4\n"
                least_sim = find_least_sim(app1, app2app_sim_dd, job_list) # return value is a tuple (appName, distVal)
                TARGET = least_sim[0]
                #print("step 4 => Found: {}".format(TARGET))

                #
                # update appPool by removing the selected app to corun
                # 
                #print("=> run {} with {}".format(app1, TARGET))
                idx = appPool.index(TARGET)
                del appPool[idx]

                run2_list.append([app1,TARGET])

        #print appPool

        #=================================#
        # end of checking appPool
        #=================================#
    return run2_list

In [59]:
app_seq1 = np.load('../app_seq1.npy')
print app_seq1

appPool = [i for i in app_seq1]

run2_list = pick_app2run(appPool, app2newfeat_dd, app_iobound_dd, app2app_sim_dd)

run2_mat1 = np.array(run2_list)
np.save('run2_mat1.npy', run2_mat1)

[u'rodinia_lud' u'cudasdk_transpose' u'cudasdk_simpleCUFFTcallback'
 u'lonestar_dmr' u'cudasdk_BlackScholes' u'cudasdk_dct8x8' u'parboil_lbm'
 u'cudasdk_convolutionSeparable' u'shoc_lev1md5hash'
 u'cudasdk_MCSingleAsianOptionP' u'rodinia_pathfinder' u'poly_fdtd2d'
 u'cudasdk_radixSortThrust' u'cudasdk_segmentationTreeThrust' u'poly_3mm'
 u'parboil_stencil' u'cudasdk_lineOfSight' u'rodinia_heartwall'
 u'rodinia_backprop' u'cudasdk_FDTD3d' u'cudasdk_eigenvalues'
 u'cudasdk_c++11Cuda' u'cudasdk_threadFenceReduction' u'poly_correlation'
 u'poly_atax' u'cudasdk_SobolQRNG' u'rodinia_hybridsort'
 u'cudasdk_shflscan' u'parboil_mriq' u'shoc_lev1sort' u'parboil_bfs'
 u'cudasdk_dxtc' u'shoc_lev1fft' u'cudasdk_scalarProd' u'poly_bicg'
 u'lonestar_mst' u'poly_covariance' u'cudasdk_convolutionFFT2D'
 u'poly_syr2k' u'cudasdk_scan' u'parboil_sgemm' u'cudasdk_vectorAdd'
 u'lonestar_sssp' u'rodinia_dwt2d' u'cudasdk_matrixMul' u'rodinia_hotspot'
 u'cudasdk_mergeSort' u'rodinia_b+tree' u'cudasdk_MCEstimat

In [58]:
# run2_seq1 = np.load('run2_mat1.npy')
# run2_seq1.shape

(39, 2)

In [60]:
app_seq = np.load('../app_seq2.npy')
print app_seq

appPool = [i for i in app_seq]

run2_list = pick_app2run(appPool, app2newfeat_dd, app_iobound_dd, app2app_sim_dd)

run2_mat = np.array(run2_list)
np.save('run2_mat2.npy', run2_mat)

[u'rodinia_lud' u'cudasdk_lineOfSight' u'cudasdk_c++11Cuda'
 u'cudasdk_dct8x8' u'cudasdk_reduction' u'parboil_mriq'
 u'cudasdk_simpleCUBLAS' u'cudasdk_mergeSort'
 u'cudasdk_convolutionSeparable' u'rodinia_backprop'
 u'cudasdk_MCSingleAsianOptionP' u'poly_gemm' u'cudasdk_scan'
 u'cudasdk_dwtHaar1D' u'cudasdk_concurrentKernels' u'shoc_lev1fft'
 u'cudasdk_fastWalshTransform' u'rodinia_lavaMD' u'rodinia_hotspot'
 u'poly_bicg' u'cudasdk_sortingNetworks' u'parboil_sgemm'
 u'cudasdk_matrixMul' u'cudasdk_interval' u'cudasdk_transpose'
 u'poly_gesummv' u'cudasdk_convolutionFFT2D'
 u'cudasdk_quasirandomGenerator' u'shoc_lev1sort' u'cudasdk_batchCUBLAS'
 u'cudasdk_convolutionTexture' u'shoc_lev1GEMM' u'rodinia_heartwall'
 u'cudasdk_shflscan' u'poly_3dconv' u'cudasdk_segmentationTreeThrust'
 u'poly_mvt' u'shoc_lev1md5hash' u'cudasdk_BlackScholes' u'cudasdk_dxtc'
 u'cudasdk_boxFilterNPP' u'poly_fdtd2d' u'cudasdk_scalarProd'
 u'rodinia_gaussian' u'poly_syrk' u'cudasdk_stereoDisparity' u'poly_2dconv'

In [61]:
app_seq = np.load('../app_seq3.npy')
#print app_seq

appPool = [i for i in app_seq]

run2_list = pick_app2run(appPool, app2newfeat_dd, app_iobound_dd, app2app_sim_dd)

run2_mat = np.array(run2_list)
np.save('run2_mat3.npy', run2_mat)